# **🧬 생물정보학과 파이썬 💻**

* <img src="https://microbenotes.com/wp-content/uploads/2018/12/Bioinformatics-Introduction-and-Applications.jpg">

## Chapter 6: 바이오파이썬 — 중심원리 (DNA → RNA → Protein)

### 🧬 중심원리를 코드로 구현해보자~!
실제 NCBI 염기서열을 **Entrez.efetch**로 불러와 **복제(상보/역상보)**, **전사(mRNA)**, **번역(ORF→단백질)** 을 진행합니다.  


---

### 🎯 실습 목표
- 🧮 NCBI에서 실제 **유전자/전사체 서열**을 `Entrez.efetch`로 가져옵니다.  
- 💾 DNA에서 **상보/역상보 가닥**, **GC%** 를 계산합니다.  
- 🔗 DNA를 전사하여 **mRNA**를 얻고, **코돈 단위**로 확인합니다.  
- 🧬 mRNA에서 **ATG로 시작하는 ORF**를 찾아 **STOP 전까지 번역**합니다.  

---

### 🔧 바이오파이썬의 편의성
- 🧪 **실험 데이터 자동화**: 여러 샘플의 서열을 일괄 전사·번역하여 분석 효율을 높입니다.  
- ⚖️ **변이 영향 파악**: 한 염기 치환이 **코돈/아미노산**에 주는 변화를 즉시 확인합니다.  
- 🔁 **확장성**: 접근번호만 바꿔 다른 **유전자/종**에도 손쉽게 적용됩니다.  

---


```python

## 0) 준비하기 — 라이브러리 설치 & 설정

### 💻 Biopython 설치하기
바이오파이썬(Biopython)은 생물학 데이터를 다루기 위한 대표적인 파이썬 라이브러리입니다.  
`Entrez`, `SeqIO`, `Seq` 등을 활용해 유전자 서열을 불러오고 처리할 수 있습니다.

---

```python
!pip install biopython


In [12]:
# Biopython은 생물정보학 데이터를 다루기 위한 대표적인 파이썬 라이브러리입니다.
!pip install biopython

In [13]:
# Entrez : NCBI 데이터베이스(유전자·단백질·논문 등)에 접근할 때 사용
# SeqIO  : 염기서열 데이터(FASTA, GenBank 등)를 불러오고 저장할 때 사용

from Bio import Entrez
from Bio import SeqIO
Entrez.email = "cluid0416@naver.com" # 실제 이메일 주소로 변경해주세요.

```python

🔗 [NCBI (Homo sapiens hemoglobin subunit beta mRNA)](https://www.ncbi.nlm.nih.gov/nuccore/NM_000518.5)

In [14]:
# RefSeq 접근번호: 인간 헤모글로빈 β 유전자 mRNA (NM_000518.5)
handle = Entrez.efetch(
    db="nucleotide",       # 염기서열 데이터베이스
    id="NM_000518.5",      # HBB 유전자 RefSeq ID
    rettype="fasta",       # 출력 형식
    retmode="text"         # 텍스트 모드로 읽기
)

# fasta 데이터 변수 지정
sequence = SeqIO.read(handle, "fasta")

# 데이터 출력
print("📄 불러온 서열 ID:", sequence.id)
print("📏 길이(bp):", len(sequence.seq))
print("🧬 앞 100nt:", sequence.seq[:100])

📄 불러온 서열 ID: NM_000518.5
📏 길이(bp): 628
🧬 앞 100nt: ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGAGGAGAAGTCTGCCGTTACTGCCCTGTGGGG


In [50]:
# 복제 단계: 상보(complement), 역상보(reverse_complement), GC% 계산

from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction

# DNA 서열 정규화 (혹시 U가 있다면 T로 통일)
hbb_seq = sequence.seq.replace("U", "T")
hbb_slice = hbb_seq[0:150]


# 상보적(complement) 서열 생성
comp_seq = hbb_slice.complement()

# 역상보(reverse complement) 서열 생성
rev_comp_seq = hbb_slice.reverse_complement()

print(f"원본   서열: 5'-{hbb_slice}-3'")
print(f"상보   서열: 3'-{comp_seq}-5'")
print(f"역상보 서열: 5'-{rev_comp_seq}-3'")



원본   서열: 5'-ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGAGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGCTGCTGG-3'
상보   서열: 3'-TGTAAACGAAGACTGTGTTGACACAAGTGATCGTTGGAGTTTGTCTGTGGTACCACGTAGACTGAGGACTCCTCTTCAGACGGCAATGACGGGACACCCCGTTCCACTTGCACCTACTTCAACCACCACTCCGGGACCCGTCCGACGACC-5'
역상보 서열: 5'-CCAGCAGCCTGCCCAGGGCCTCACCACCAACTTCATCCACGTTCACCTTGCCCCACAGGGCAGTAACGGCAGACTTCTCCTCAGGAGTCAGATGCACCATGGTGTCTGTTTGAGGTTGCTAGTGAACACAGTTGTGTCAGAAGCAAATGT-3'


In [47]:
# GC 함량 계산
gc_percent = gc_fraction(hbb_slice) * 100
print(f"GC 함량: {gc_percent:.2f}%")


GC 함량: 54.50%


```python

In [51]:
# 전사 단계: DNA → mRNA (T → U)

# DNA 조각(hbb_slice)을 전사하여 mRNA 생성
hbb_mrna = hbb_slice.transcribe()

# 출력
print(f"DNA 서열 : {hbb_slice}")
print(f"mRNA 서열: {hbb_mrna}")



DNA 서열 : ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGAGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGCTGCTGG
mRNA 서열: ACAUUUGCUUCUGACACAACUGUGUUCACUAGCAACCUCAAACAGACACCAUGGUGCAUCUGACUCCUGAGGAGAAGUCUGCCGUUACUGCCCUGUGGGGCAAGGUGAACGUGGAUGAAGUUGGUGGUGAGGCCCUGGGCAGGCUGCUGG


In [52]:
# 번역 단계: mRNA → 단백질

# mRNA를 단백질로 번역 (STOP 코돈에서 자동 종료)
protein = hbb_mrna.translate(to_stop=True)


# 결과 출력
print(f"mRNA 서열 : {hbb_mrna}")
print(f"단백질 서열: {protein}")
print(f"단백질 길이: {len(protein)}개 아미노산")


mRNA 서열 : ACAUUUGCUUCUGACACAACUGUGUUCACUAGCAACCUCAAACAGACACCAUGGUGCAUCUGACUCCUGAGGAGAAGUCUGCCGUUACUGCCCUGUGGGGCAAGGUGAACGUGGAUGAAGUUGGUGGUGAGGCCCUGGGCAGGCUGCUGG
단백질 서열: TFASDTTVFTSNLKQTPWCI
단백질 길이: 20개 아미노산


```python

In [56]:
# 전체 번역: STOP 코돈 이후도 포함 (to_stop=False)

protein_full = hbb_mrna.translate(to_stop=False)

# 결과 출력
print(f"mRNA 서열 : {hbb_mrna}")
print(f"전체 단백질 서열: {protein_full}")
print(f"단백질 길이(※ STOP 포함): {len(protein_full)}개 아미노산")

# STOP 코돈 개수 확인
print("STOP(*) 개수:", str(protein_full).count("*"))


mRNA 서열 : ACAUUUGCUUCUGACACAACUGUGUUCACUAGCAACCUCAAACAGACACCAUGGUGCAUCUGACUCCUGAGGAGAAGUCUGCCGUUACUGCCCUGUGGGGCAAGGUGAACGUGGAUGAAGUUGGUGGUGAGGCCCUGGGCAGGCUGCUGG
전체 단백질 서열: TFASDTTVFTSNLKQTPWCI*LLRRSLPLLPCGAR*TWMKLVVRPWAGCW
단백질 길이(※ STOP 포함): 50개 아미노산
STOP(*) 개수: 2


============================================================
## CHAPTER 6 정리: 중심원리 (DNA → RNA → Protein)
============================================================

### ✅ 배운 내용
- **NCBI Entrez.efetch**를 이용해 실제 유전자 서열을 불러올 수 있다.  
- DNA의 **상보(complement)**, **역상보(reverse complement)**, **GC 함량**을 계산할 수 있다.  
- DNA를 전사하여 **mRNA 서열**을 얻을 수 있다.  
- mRNA를 번역하여 **단백질 아미노산 서열**로 변환할 수 있다.  
- **Biopython의 내장 메서드**(`complement()`, `transcribe()`, `translate()`)만으로 중심원리를 구현할 수 있다.  

---

### 🧬 생명과학 응용
-  **유전자 기능 예측** → 전사·번역 과정을 통해 단백질 기능 탐색  
-  **돌연변이 분석** → 염기 치환이 단백질 서열에 주는 영향 비교  
-  **유전체 자동 분석** → Biopython으로 중심원리 과정을 자동화